In [2]:
import os
from collections import defaultdict
import sys
import pickle
import numpy as np
from numpy import gradient
from scipy.interpolate import interp1d
from math import pi, ceil
from packages import data_container
from packages.data_container import Data
from packages.helper import traj_speed, beta, min_dist, d_psi, min_sep, d_theta
import matplotlib.pyplot as plt
# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container

In [4]:
'''Import data from csv to Data object'''
header=('subj_x', 'subj_y', 'subj_z', 'subj_yaw', 'subj_pitch', 'subj_row')
info={'p_goal':[], 'p_obst':[], 'obst_angle':[], 'obst_speed':[], 'subj_id':[], 'trial_id':[], 'obst_onset':[], 
      'decision_onset':[], 'threshold_onset':[], 'obst_out':[], 'w_goal': 0.4, 'w_obst': 0.2, 'ps_trial':[], 
      'ps_subj':[], 'time_stamp':[], 'pass_order':[], 'p_subj':[], 'v_subj':[], 'v_obst':[], 'a_subj':[], 
      'rel_dpsi': [], 'dtheta': []}
info['dpsi_threshold'] = 0.04
# The raw data is 90 Hz
data = Data(Hz=90, header=header, info=info)

folder = os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Bai_movObst1')
os.listdir(folder)

for file in os.listdir(folder):
    # Import output
    file_path = os.path.abspath(os.path.join(folder, file))
    subj_id = int(file[-6:-4])
    with open(file_path, 'r') as f:
        trials = [line.split(',') for line in f.read().split('\n')]
    head = 0
    while head + 1 < len(trials):
        for i in range(head+1, len(trials)):
            if trials[i][0] == '':
                tail = i - 1
                break
        trial_id = int(trials[head+1][1])
        time_stamp = [float(line[0]) for line in trials[head+3:tail+1]]
        angle = float(trials[head+1][3])
        speed = float(trials[head+1][5])
        subj = np.array([[float(val) for val in line[1:7]] for line in trials[head+3:tail+1]])
        p0 = Data.filter(subj[:,[0,2]], time_stamp, 90, 4, 0.6)
        v0 = gradient(p0, axis=0) * data.Hz
        a0 = gradient(v0, axis=0) * data.Hz
        goal = np.array([[float(line[-2]), float(line[-1])] for line in trials[head+3:tail+1]])
        obst = np.array([[float(line[-4]), float(line[-3])] for line in trials[head+3:tail+1]])
        onset = obst_out = decision_onset = threshold_onset = None
        rel_dpsi = []
        dtheta = []
        if speed:
            for i in range(len(obst) - 1):
                if obst[i][0] != obst[i + 1][0] or obst[i][1] != obst[i + 1][1]:
                    onset = i
                    break
            obst[onset:] = np.linspace(obst[onset], obst[-1], len(obst[onset:]))
            for i in range(onset, len(subj) - 1):
                _beta = beta(p0[i], obst[i], v0[i])
                if abs(_beta) > pi / 2:
                    obst_out = i
                    # 1: pass in front of obst, -1: pass behind obst, 0: freewalk trial
                    pass_order = np.sign(_beta * -angle)
                    break
            v1 = (obst[-1] - obst[onset]) / (len(obst) - 1 - onset) * data.Hz
            v1 = np.tile(v1, (len(obst),1))
            # Compute -psi*sign(obst_angle)
            rel_dpsi = np.zeros(len(p0))
            dthetas = np.zeros(len(p0))
            t0, t1 = onset, obst_out
            rel_dpsi[t0:t1] = -d_psi(p0[t0:t1], obst[t0:t1], v0[t0:t1], v1[t0:t1]) * np.sign(angle)
            dthetas[t0:t1] = d_theta(p0[t0:t1], obst[t0:t1], v0[t0:t1], v1[t0:t1], w=data.info['w_obst'])
            decision_onset = threshold_onset = None
            if abs(angle) != 180:
                mi = np.argmin(np.absolute(rel_dpsi[t0:t1]))
                for t in range(t1 - 1, mi - 1, -1):
                    if abs(rel_dpsi[t]) >= info['dpsi_threshold']:
                        threshold_onset = t
                for t in range(t1 - 1, t0 - 1, -1):
                    if np.sign(rel_dpsi[t]) != pass_order:
                        decision_onset = t + 1
                        break
                else:
                    decision_onset = onset
        else:
            pass_order = 0  # 1: pass in front of obst, -1: pass behind obst, 0: freewalk trial
        data.add_traj(subj)
        ps_trial = np.mean(traj_speed(subj[-2*data.Hz:-data.Hz,[0,2]], data.Hz))
        data.add_info({'p_goal':goal, 'p_obst':obst, 'obst_angle':angle, 'obst_speed':speed, 'subj_id':subj_id, 
                       'trial_id':trial_id, 'obst_onset':onset, 'obst_out':obst_out, 'time_stamp':time_stamp, 
                       'ps_trial':ps_trial, 'ps_subj':0, 'pass_order':pass_order, 'decision_onset':decision_onset,
                       'threshold_onset':threshold_onset, 'p_subj':p0, 'v_subj':v0, 'a_subj':a0, 'v_obst':v1,
                       'rel_dpsi': rel_dpsi, 'dtheta': dthetas})
        head = tail + 1

'''Dump bottom 1% trials that have the smallest minimal speed'''
vs = []
for i in range(len(data.trajs)):
    if i in data.dump or data.info['obst_speed'][i] == 0:
        continue
    t0 = data.info['obst_onset'][i]
    t1 = data.info['obst_out'][i]
    s = min(traj_speed(data.info['p_subj'][i][t0:t1], Hz=data.Hz))
    vs.append((s, i))
vs.sort()
n = int(len(vs) / 100) # 1% of trials
for i in [x[1] for x in vs[:n]]:
    data.dump[i] = 'min speed below bottom 1%'
    
ps_subj = defaultdict(int)
for i in range(len(data.trajs)):
    if data.info['obst_speed'][i] == 0:
        subj_id = data.info['subj_id'][i]
        ps_subj[subj_id] += data.info['ps_trial'][i] / 10
for i in range(len(data.trajs)):
    subj_id = data.info['subj_id'][i]
    data.info['ps_subj'][i] = ps_subj[subj_id]

outfile = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Bai_movObst1_data.pickle'))       
with open(outfile, 'wb') as f:   
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [5]:
'''Save a 30 Hz version for fitting'''
# Convert to 30 Hz for fitting
data.Hz = 30
for i in range(len(data.trajs)):
    data.trajs[i] = data.trajs[i][::3]
    data.info['p_goal'][i] = data.info['p_goal'][i][::3]
    data.info['p_obst'][i] = data.info['p_obst'][i][::3]
    data.info['time_stamp'][i] = data.info['time_stamp'][i][::3]
    data.info['p_subj'][i] = data.info['p_subj'][i][::3]
    data.info['v_subj'][i] = data.info['v_subj'][i][::3]
    data.info['a_subj'][i] = data.info['a_subj'][i][::3]
    data.info['v_obst'][i] = data.info['v_obst'][i][::3]
    if data.info['obst_onset'][i]:
        data.info['obst_onset'][i] = ceil(data.info['obst_onset'][i] / 3)
        data.info['obst_out'][i] = data.info['obst_out'][i] // 3
    if data.info['decision_onset'][i]:
        data.info['decision_onset'][i] = ceil(data.info['decision_onset'][i] / 3)
    if data.info['threshold_onset'][i]:
        data.info['threshold_onset'][i] = ceil(data.info['threshold_onset'][i] / 3)
    if len(data.info['rel_dpsi'][i]) > 0:
        data.info['rel_dpsi'][i] = data.info['rel_dpsi'][i][::3]
    if len(data.info['dtheta'][i]) > 0:
        data.info['dtheta'][i] = data.info['dtheta'][i][::3]
outfile = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Bai_movObst1_data_30Hz.pickle'))       
with open(outfile, 'wb') as f:   
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [44]:
length = []
for i in range(len(data.trajs)):    
    if data.info['obst_out'][i]:
        length.append((data.info['obst_out'][i] - data.info['obst_onset'][i], i, data.info['subj_id'][i], data.info['trial_id'][i]))
length.sort()
length[-50:]
len(length)
len(data.trajs) / 12

160.0

In [45]:
'''check redundant trials'''
trials = defaultdict(int)
for i in range(len(data.trajs)):
    subj_id = data.info['subj_id'][i]
    trial_id = data.info['trial_id'][i]
    trials[(subj_id, trial_id)] += 1
for k, v in trials.items():
    if v > 1:
        print(k, v)

In [25]:
set(data.info['subj_id'])

{0, 1, 2, 3, 4, 6, 7, 9, 10, 11, 12, 13}

In [ ]:
'''Check different start time'''
for i in range(100):
    t0 = data.info['obst_onset'][i]
    t0_match = data.info['stimuli_match'][i]
    t1 = data.info['obst_out'][i]
    if t0 != t0_match:
        print(t0, t0_match, t1)

In [ ]:
'''Check matching rate'''
for i in range(1400):
    if not data.info['obst_speed'][i]:
        continue
    pass_order = data.info['pass_order'][i]
    t0 = data.info['obst_onset'][i]
    t0_match = data.info['stimuli_match'][i]    
    p0 = data.info['p_subj'][i][t0_match]
    v0 = data.info['v_subj'][i][t0_match]
    p1 = data.info['p_obst'][i][t0_match]
    v1 = data.info['v_obst'][i]
    order_pred = np.sign(beta(p0, p1, v0) * d_psi(p0, p1, v0, v1))
    if pass_order * order_pred < 0:
        print('subj ', data.info['subj_id'][i], 'trial ', data.info['trial_id'][i], 
              'angle ', data.info['obst_angle'][i],
              pass_order, order_pred, beta(p0, p1, v0) * d_psi(p0, p1, v0, v1))